In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import spark_partition_id
from pyspark.sql import functions as sf

In [3]:
spark = SparkSession.builder.master("local[2]").appName("Vprople").getOrCreate()

In [4]:
filepath = "../data/propel_department*.csv"
department = spark.read.format("csv").option("path",filepath).option("inferSchema","true").option("header","true").load()

In [5]:
department.show()

+--------+---------+--------+
|deptcode| deptname|location|
+--------+---------+--------+
|     101|       IT|   Noida|
|     102|       HR|   Delhi|
|     103|    Sales|   Noida|
|     104|Marketing|   Noida|
+--------+---------+--------+



In [6]:
filepath = "../data/vpropel_employee*.csv"
employee = spark.read.format("csv").option("path",filepath).option("inferSchema","true").option("header","true").load()
employee.show()

+-------+---------+--------+-------------------+----------+-------------+-------+----------+--------+
|empcode| empfname|emplname|              email|   phoneno|          job|manager|  hiredate|deptcode|
+-------+---------+--------+-------------------+----------+-------------+-------+----------+--------+
|      1|   Taukir|    Khan|   taukir@gmail.com|8010339935|Data Engineer|Santosh|2021-10-27|     101|
|      2|   Saheba|  pathan|   saheba@gmail.com|3020339935|       DevOps|Santosh|2022-10-27|     101|
|      3|   Zeenat|    khan|   zeenat@gmail.com|2015339935|       Tester|Santosh|2022-06-20|     101|
|      5|     Sonu|    sona|     sonu@gmail.com|9115339935|     Staffing| Anajan|2020-07-20|     102|
|      6|    Rahul|     cho|    rahul@gmail.com|8915339935|   advertiser|   Hema|2020-02-20|     103|
|      7|    lavan|   reddy|    lavan@gmail.com|2115339935|      calling|   Hema|2021-02-20|     103|
|      4|      Sab|   Saikh|   taukir@gmail.com|7015339935|    Recruiter| Anajan|2

In [8]:
filepath = "../data/vpropel_salary*.csv"
salary = spark.read.format("csv").option("path",filepath).option("inferSchema","true").option("header","true").load()
salary.show()

+-------+-------+---------+----------+
|empcode|  basic|additions|deductions|
+-------+-------+---------+----------+
|      1|15000.0|    500.0|       0.0|
|      2|10000.0|    500.0|       0.0|
|      3| 8000.0|    300.0|       0.0|
|      4| 6000.0|    500.0|       0.0|
|      5|10000.0|    500.0|       0.0|
|      6| 5000.0|   2000.0|       0.0|
|      7| 6000.0|   1500.0|     500.0|
+-------+-------+---------+----------+



## Ambiguty

In [30]:
employee.join(department, employee['deptcode'] == department['deptcode'], "inner").\
drop(department.deptcode).\  # Drop similar colum or renamed before join
select("empfname","deptname","deptcode").show()

SyntaxError: unexpected character after line continuation character (350273423.py, line 2)

# 1: find out duplicate email address from emplyee table;

In [10]:
employee.groupBy("email").count().where("count > 1").show()

+----------------+-----+
|           email|count|
+----------------+-----+
|taukir@gmail.com|    2|
+----------------+-----+



## Display DeptName, and number of employee in each department.

In [17]:
employee.join(department, employee['deptcode'] == department['deptcode'], "inner").\
groupBy(sf.col("deptname")).count().alias("num_employees").show()

+--------+-----+
|deptname|count|
+--------+-----+
|   Sales|    2|
|      HR|    2|
|      IT|    3|
+--------+-----+



## find the department_name and  total spend salary on employee  by each department.

In [15]:
employee.join(department, employee['deptcode'] == department['deptcode']).\
join(salary,employee['empcode'] == salary['empcode']).\
groupBy(sf.col("deptname")).agg(sf.sum(sf.expr("basic + additions - deductions")).alias("batch_bal")).show()

+--------+---------+
|deptname|batch_bal|
+--------+---------+
|   Sales|  14000.0|
|      HR|  17000.0|
|      IT|  34300.0|
+--------+---------+

